In [6]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [5]:
print("msno")
msno_values = data.msno.unique()
print("end msno")
del(data)

msno
end msno


In [85]:
user_logs = pd.read_csv('raw_data/user_logs.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
user_logs_2 = pd.read_csv('raw_data/user_logs_v2.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
#user_logs = pd.concat((user_logs, user_logs_2))

KeyboardInterrupt: 

In [ ]:
user_logs.head()

In [ ]:
counts = user_logs["msno"].value_counts()
counts.to_csv("cleaned_input/counts.csv")

In [141]:
del(user_logs)
del(user_logs_2)

NameError: name 'user_logs' is not defined

In [7]:
counts = pd.read_csv("cleaned_input/counts.csv")

In [8]:
#get number of unique visits
user_features = pd.DataFrame() 

user_features['msno'] = np.nan #counts.index
#user_features["frequency"] = np.nan #counts.values
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29


In [13]:
#read in user_logs
import time
chunksize = 25000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    start_time = time.time()
    chunk = chunk[chunk['msno'].isin(msno_values)]
    rows = chunk.msno.unique()
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    #print("processing ",i,"took %s seconds" % (time.time() - start_time))
    #print(len(rows))
    for row in rows:
        row_time = time.time()
        df = chunk[chunk["msno"] == row]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        differences = df.date.diff()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gap_days = differences.sum()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        average_gap_days = differences.mean()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        if len(differences.nlargest(1)) > 0:
            longest_gap = differences.nlargest(1).iloc[0].total_seconds()
        else:
            longest_gap = -1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
        print(user_features.head())
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
    #del(chunk)
    i += 1 
    print("Round ",i,"took %s seconds" % (time.time() - start_time))
    if i % 100 == 0:
        user_features.to_csv(str(i) + "_user_logs_features.csv")
user_features.head()

                                           msno gap_days  average_gap_days  \
0  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8= 732 days      5.270400e+06   
1  PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g= 402 days      1.736640e+07   
2  KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8= 532 days      1.532160e+07   
3  OcZ3fKQzHrL1BuzuFRzUiwPr8l9P8JKCFpafdYf5sCY= 689 days      5.411782e+06   
4  dW5DQO92XaG024D2QYmktyUnLc2axil74zUPAhF8sEs= 673 days      5.814720e+06   

   longest_gap  gaps_longer_29  
0    7430400.0            12.0  
1   18144000.0             2.0  
2   25747200.0             3.0  
3    6652800.0            11.0  
4   10627200.0            10.0  
                                           msno gap_days  average_gap_days  \
0  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8= 732 days      5.270400e+06   
1  PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g= 402 days      1.736640e+07   
2  KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8= 532 days      1.532160e+07   
3  OcZ3fKQzHrL1BuzuFR

KeyboardInterrupt: 

In [161]:
user_features.to_csv("cleaned_input/user_logs_features_2.csv")

In [ ]:
chunksize = 50000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs_2.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    start_time = time.time()
    chunk = chunk[chunk['msno'].isin(msno_values)]
    rows = chunk.msno.unique()
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    #print("processing ",i,"took %s seconds" % (time.time() - start_time))
    #print(len(rows))
    for row in rows:
        row_time = time.time()
        df = chunk[chunk["msno"] == row]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        differences = df.date.diff()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gap_days = differences.sum()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        average_gap_days = differences.mean()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        if len(differences.nlargest(1)) > 0:
            longest_gap = differences.nlargest(1).iloc[0].total_seconds()
        else:
            longest_gap = -1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
    #del(chunk)
    i += 1 
    print("Round ",i,"took %s seconds" % (time.time() - start_time))
user_features.head()

In [ ]:
#combine duplicates? 

In [ ]:
user_features.head()

In [ ]:
user_features.to_csv("user_logs_features.csv")

In [ ]:
#combine what you 